This is an end-to-end pipeline for pulling and cleaning lightcurve data, passing it through an autoencoder and extracting the latent space, and performing clustering on the latent space.

A document showing some sample lightcurves for each cluster can be found in the README.

**CLEANING DATA**

Please only use one of the following two cells' cleaning methods; both will create a final folder with each lightcurve's cleaned data. 

In [ ]:
# Normalizing the bin size 
# Uses both number of bins and bin_size to ensure that as a parameter instead of bin size

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pickle

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)

# location on desktop for file to be saved to
path_pickle = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves_Pickle/" 
path = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves/"
path_clean = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves_Clean/"
        
for x in range(0, 4000):
    print(x)
    try:
        TOI_name = "TIC " + rows[x][fields.index("TIC ID")] 
        period = float(rows[x][fields.index("Period (days)")])
        t0 = float(rows[x][fields.index("Epoch (BJD)")])
        duration_hours = float(rows[x][fields.index("Duration (hours)")])
        fractional_duration = (duration_hours / 24.0) / period
        
        #pulling authors in order of preference
        search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
        author = "SPOC"
        if(len(search_result) == 0):
            search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
            author = "TESSSPOC"
            if(len(search_result) == 0):
                search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                author = "QLP"

        lcs = search_result.download_all() 
        lc_raw = lcs.stitch() #stitching different sectors of data 

        lc_clean = lc_raw #removed the remove_outliers operation due to unknown error?

        temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000)) #fold lightcurve by phase

        fractional_duration = (duration_hours / 24.0) / period
        
        if(duration_hours < 3.0): #changing window of data based on duration hours and fractional duration
            
            if(fractional_duration < 0.004): 

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*300*fractional_duration/301, n_bins=301)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            if(fractional_duration < 0.01):

                phase_mask = (temp_fold.phase > -15*fractional_duration) & (temp_fold.phase < 15.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*15*fractional_duration/301, n_bins=301) 
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            elif(fractional_duration < 0.04):

                phase_mask = (temp_fold.phase > -4*fractional_duration) & (temp_fold.phase < 4.0*fractional_duration) 
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*4*fractional_duration/301, n_bins=301)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
            
            else:

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*15*fractional_duration/301, n_bins=301)  
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
    
        elif(duration_hours < 5.0):

            phase_mask = (temp_fold.phase > -25*fractional_duration) & (temp_fold.phase < 25.0*fractional_duration) 
            lc_zoom = temp_fold[phase_mask]

            lc_local = lc_zoom.bin(time_bin_size=2*25*fractional_duration/301, n_bins=301)
            lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
        
        else:
            
            if(fractional_duration < 0.02):

                phase_mask = (temp_fold.phase > -300*fractional_duration) & (temp_fold.phase < 300.0*fractional_duration)
                lc_zoom = temp_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*300*fractional_duration/301, n_bins=301)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
                
            else:

                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

                lc_flat = lc_clean.flatten(mask=transit_mask)

                lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

                phase_mask = (lc_fold.phase > -15*fractional_duration) & (lc_fold.phase < 15.0*fractional_duration) 
                lc_zoom = lc_fold[phase_mask]

                lc_local = lc_zoom.bin(time_bin_size=2*15*fractional_duration/301, n_bins=301)
                lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
        
        #saving each lightcurve's data to a new .txt file
        fig, axes = plt.subplots()
        lc_local.plot(ax=axes, color="black")
        line = fig.gca().lines[0]
        data = np.asarray([line.get_xdata(), line.get_ydata()])
        filename = path_clean + "fig" + str(x) + ".csv"
        np.savetxt(filename, data, delimiter=',')
        plt.close()
        
    except AttributeError:
        print("AttributeError occured with", TOI_name, "item", str(x))
    except ZeroDivisionError:
        print("ZeroDivisionError occured with", TOI_name, "item", str(x))
    except:
        print("An error occured with", TOI_name, "item", str(x))

In [ ]:
# Normalizing the bin size 
# Uses both number of bins and bin_size to ensure that as a parameter instead of bin size
# Using two times the fractional duration for the width of the transit, as opposed to variable widths like in the previous method

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pickle

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)

path_pickle = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves_Pickle/"
path = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves/"
path_clean = "/Users/anahitasrinivasan/Desktop/UROP Items/Lightcurves_Clean/"
        
for x in range(0, 10):
    print(x)
    try:
        TOI_name = "TIC " + rows[x][fields.index("TIC ID")] 
        period = float(rows[x][fields.index("Period (days)")])
        t0 = float(rows[x][fields.index("Epoch (BJD)")])
        duration_hours = float(rows[x][fields.index("Duration (hours)")])
        fractional_duration = (duration_hours / 24.0) / period
        
        search_result = lk.search_lightcurve(TOI_name, author="SPOC", exptime=120)
        author = "SPOC"
        if(len(search_result) == 0):
            search_result = lk.search_lightcurve(TOI_name, author="TESS-SPOC", exptime=1800)
            author = "TESSSPOC"
            if(len(search_result) == 0):
                search_result = lk.search_lightcurve(TOI_name, author="QLP", exptime=1800)
                author = "QLP"

        lcs = search_result.download_all() 
        lc_raw = lcs.stitch() 

        lc_clean = lc_raw

        temp_fold = lc_clean.fold(period, epoch_time=(t0-2457000))

        fractional_duration = (duration_hours / 24.0) / period
        
        phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 2.0)
        transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])

        lc_flat = lc_clean.flatten(mask=transit_mask)

        lc_fold = lc_flat.fold(period, epoch_time=(t0-2457000))

        phase_mask = (lc_fold.phase > -2*fractional_duration) & (lc_fold.phase < 2.0*fractional_duration) 
        lc_zoom = lc_fold[phase_mask]

        lc_local = lc_zoom.bin(time_bin_size=2*15*fractional_duration/301, n_bins=301)  
        lc_local = (lc_local - np.abs(np.nanmin(lc_local.flux))) / (np.abs(np.nanmax(lc_local.flux)) - np.abs(np.nanmin(lc_local.flux)))
        
        fig, axes = plt.subplots()
        lc_local.scatter(ax=axes, color="black")
        line = fig.gca().lines[0]
        data = np.asarray([line.get_xdata(), line.get_ydata()])
        filename = path_clean + "fig" + str(x) + ".csv"
        np.savetxt(filename, data, delimiter=',')
        plt.close()
        
    except AttributeError:
        print("AttributeError occured with", TOI_name, "item", str(x))
    except ZeroDivisionError:
        print("ZeroDivisionError occured with", TOI_name, "item", str(x))
    except:
        print("An error occured with", TOI_name, "item", str(x))

**CREATING HISTOGRAMS**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pickle

data = "finaldata.csv"
fields = []
rows = []

with open(data, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)
        
periods_raw = []
periods_to_100 = []
periods_to_20 = []
duration_hours = []
fractional_durations = []

for row in rows:
    period = float(row[fields.index("Period (days)")])
    duration = float(row[fields.index("Duration (hours)")])
    if period != 0:
        fractional_duration = (duration / 24.0) / period
    
    if(period <= 100):
        periods_to_100.append(period)
    if(period <= 20):
        periods_to_20.append(period)
    periods_raw.append(period)
    duration_hours.append(duration)
    fractional_durations.append(fractional_duration)

fig1, axs1 = plt.subplots(1, 1) 
axs1.set_title("Periods (Days)")
axs1.set_xlabel('Period (Days)')
axs1.set_ylabel('Number of TOIs')
axs1.hist(periods_raw, bins=50)
fig1.savefig("periodhistogram.png")

fig2, axs2 = plt.subplots(1, 1) 
axs2.set_title("Periods (Days) Zoomed in on 100 Days")
axs2.set_xlabel('Period (Days)')
axs2.set_ylabel('Number of TOIs')
axs2.hist(periods_to_100, bins=50)
fig2.savefig("periodhistogram100days.png")

fig3, axs3 = plt.subplots(1, 1)
axs3.set_title("Periods (Days) Zoomed in on 20 Days")
axs3.set_xlabel('Period (Days)')
axs3.set_ylabel('Number of TOIs')
axs3.hist(periods_to_20, bins=50)
fig3.savefig("periodhistogram20days.png")

fig4, axs4 = plt.subplots(1, 1)
axs4.set_title("Duration Hours")
axs4.set_xlabel('Duration (Hours)')
axs4.set_ylabel('Number of TOIs')
axs4.hist(duration_hours, bins=50)
fig4.savefig("durationhourshistogram.png")

fig5, axs5 = plt.subplots(1, 1)
axs5.set_title("Fractional Duration")
axs5.set_xlabel('Fractional Duration')
axs5.set_ylabel('Number of TOIs')
axs5.hist(fractional_durations, bins=50)
fig5.savefig("fractionaldurationhistogram.png")

**EXTRACTING LATENT SPACE**

At this point, I uploaded the folder with the lightcurve data to Google Drive for storage purposes.

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

In [ ]:
import csv
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
path = "/content/drive/My Drive/Lightcurves_CSVUniformBin/"
metadata_path = "/content/drive/My Drive/finaldata.csv"

all_points = []

fields = []
rows = []
with open(metadata_path, 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row)
index_location = fields.index("Depth (ppm)")

for c in range(0, 4000):
  points = []

  depth = rows[c][index_location]

  if(float(depth) >= 1000.0): # only training on lightcurves with a certain depth

    try:

      with open(path + "fig" + str(c) + ".csv", 'r') as file:

          read = csv.reader(file)
          read = list(read)
          x_data = read[0]
          y_data = read[1]
          for a in range(len(x_data)):
            x_data[a] = float(x_data[a])
            y_data[a] = float(y_data[a])

          if(len(x_data) == 301):
            
            x_data = np.divide(x_data, np.amax(x_data))
            y_data = np.array(y_data)

            for i in range(len(x_data)):
              points.append([x_data[i], y_data[i]])
            
            #imputing in for the nans - filling in with a value of -1
            imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)
            imp.fit(points)
            points = imp.transform(points)

            new_x_data = []
            new_y_data = []
            for item in points:
              new_x_data.append(item[0])
              new_y_data.append(item[1])

            y_data = np.array(new_y_data)

            array_sum = np.sum(y_data)
            array_has_nan = np.isnan(array_sum)

            if(array_has_nan == True):
              print(str(c), array_has_nan)
            else:
              all_points.append(y_data)

          else:
            print("Fig " + str(c) + " was not the required length; has length " + str(len(x_data)))

    except:
      print("An error occurred with fig", str(c))
    
  else:
    print("fig", str(c), "depth is too small; has depth", depth)

In [ ]:
sequence = np.array(all_points)
num_samples = len(sequence)
timesteps = len(sequence[0])

print(num_samples)
print(timesteps)
print(sequence.shape)

sequence = sequence.reshape(num_samples, timesteps)
print(sequence)
print(sequence.shape)

In [ ]:
original_dim = 301
intermediate_dim = 32
latent_dim = 5

inputs = keras.Input(shape=(original_dim,))
input_reshaped = layers.Reshape((original_dim, 1))(inputs)
masked = layers.Masking(mask_value=-1.0)(input_reshaped) #mask for the nans
y = layers.Conv1D(intermediate_dim, 3, activation='relu', input_shape=(original_dim, 1))(masked) #convolutional layer of size 3 to create arrays of size 298 by 32
flat = layers.Flatten()(y)
h = layers.Dense(intermediate_dim, activation='relu')(flat) 

z_mean = layers.Dense(latent_dim)(h) 
z_log_sigma = layers.Dense(latent_dim)(h)

In [ ]:
from keras import backend as K

def sampling(args): #creating a sampling function
    z_mean, z_log_sigma = args 
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1) #generating a random distribution given the parameters
    return z_mean + K.exp(z_log_sigma) * epsilon #mean + element-wise exponential

z = layers.Lambda(sampling)([z_mean, z_log_sigma]) #Lambda wraps arbitrary expressions as a layer
#uses the sampling function we just created and the arguments of z_mean and z_log_sigma from the layers before

In [ ]:
# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder') #Model maps inputs to the three outputs here

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling') #passes in the z-sampling distribution from before
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs) 
mid = layers.Dense(9568, activation='relu')(x) 
y = layers.Reshape((299, intermediate_dim))(mid)
outputs = layers.Conv1DTranspose(1, 3, activation='relu', input_shape=(intermediate_dim, 1))(y)
outputs = layers.Reshape((301,))(outputs)

decoder = keras.Model(latent_inputs, outputs, name='decoder') #maps the z-sampling to outputs of the original size of 301

# instantiate VAE model
outputs = decoder(encoder(inputs)[2]) #output is encoding the input and then decoding it again
vae = keras.Model(inputs, outputs, name='vae_mlp') #maps the original inputs to the encoded-then-decoded inputs

encoder.summary()
decoder.summary()

In [ ]:
#creating a custom loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma) #kl_loss is regularizing the latent space
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer="adam")

In [ ]:
history = vae.fit(sequence, sequence, epochs=300, batch_size=100, validation_split=0.2)

In [ ]:
#plotting loss vs. number of epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NOTE: I filter by depth for the lightcurves when training the model, but I extract the latent space of all the lightcurves. 

In [ ]:
#creating a test sequence

path = "/content/drive/My Drive/Lightcurves_CSVUniformBin/"

all_predicted_points = []

for c in range(0, 4000):
  points = []

  try:

    with open(path + "fig" + str(c) + ".csv", 'r') as file:

        read = csv.reader(file)
        read = list(read)
        x_data = read[0]
        y_data = read[1]
        for a in range(len(x_data)):
          x_data[a] = float(x_data[a])
          y_data[a] = float(y_data[a])

        if(len(x_data) == 301):
          
          x_data = np.divide(x_data, np.amax(x_data))
          y_data = np.array(y_data)

          for i in range(len(x_data)):
            points.append([x_data[i], y_data[i]])
          
          imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)
          imp.fit(points)
          points = imp.transform(points)

          new_x_data = []
          new_y_data = []
          for item in points:
            new_x_data.append(item[0])
            new_y_data.append(item[1])

          y_data = np.array([c] + new_y_data)

          array_sum = np.sum(y_data)
          array_has_nan = np.isnan(array_sum)

          if(array_has_nan == True):
            print(str(c), array_has_nan)
          else:
            np.insert(y_data, 0, c)
            # print(c)
            all_predicted_points.append(y_data)

        else:
          print("Fig " + str(c) + " was not the required length; has length " + str(len(x_data)))

  except:
    print("An error occurred with fig", str(c))

c_values = []

for i in range(len(all_predicted_points)):
  all_predicted_points[i] = all_predicted_points[i].tolist()
  c_values.append(all_predicted_points[i].pop(0))
  all_predicted_points[i] = np.array(all_predicted_points[i])

predicted_sequence = np.array(all_predicted_points)

num_samples = len(predicted_sequence)
timesteps = len(predicted_sequence[0])

predicted_sequence = predicted_sequence.reshape(num_samples, timesteps)

#extract only the latent space in 5D from each of these items
prediction = encoder.predict(predicted_sequence)

latent_size = len(prediction)
dimension = len(prediction[0])
z = len(prediction[2][0])

prediction0 = np.array(prediction[0])
prediction0 = prediction0.reshape(dimension, z)

prediction1 = np.array(prediction[1])
prediction1 = prediction1.reshape(dimension, z)

prediction2 = np.array(prediction[2])
prediction2 = prediction2.reshape(dimension, z)

In [ ]:
#Saving the latent space to another .csv file
for i in range(len(prediction0)):
  prediction0[i] = prediction0[i].tolist()
prediction0 = prediction0.tolist()

for i in range(len(prediction0)):
  prediction0[i].append(c_values[i])
prediction0 = np.array(prediction0)

pd.DataFrame(prediction0).to_csv("/content/drive/My Drive/latentspacefinal.csv")

**PERFORM CLUSTERING ON LATENT SPACE**

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
latent_space = pd.read_csv('/content/drive/My Drive/latentspace.csv')
print(latent_space.shape)

latent_space = latent_space.to_numpy()
print(latent_space.shape)

latent_space = latent_space[:,1:]
print(latent_space)
print(latent_space.shape)

In [ ]:
tsne = TSNE(n_components=2, random_state=0, verbose=1) #using TSNE to visualize the data
latent_space_2d = tsne.fit_transform(latent_space)

In [ ]:
#HDBSCAN

#!pip install hdbscan
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=15, gen_min_span_tree=True)
clusterer.fit(latent_space)
labels = clusterer.labels_

no_clusters = len(np.unique(labels))
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

# actually creating the TSNE plot using the results of the number of clusters found by HDBSCAN earlier

latent_space_2d_result_df = pd.DataFrame({'tsne_1': latent_space_2d[:,0], 'tsne_2': latent_space_2d[:,1]})
fig, ax = plt.subplots(1)

plt.scatter(latent_space_2d_result_df.tsne_1, latent_space_2d_result_df.tsne_2)

sns.scatterplot(x="tsne_1", y="tsne_2",
                palette=sns.color_palette("hls", 10), hue=labels.tolist(),
                data=latent_space_2d_result_df).set(title="Lightcurve Clustering")

In [ ]:
#FINDING SAMPLE LIGHTCURVES FOR EACH CATEGORY
import csv 

fields = []
rows = []
with open("/content/drive/My Drive/latentspacefinal.csv", 'r') as file:
    read = csv.reader(file)
    fields = next(read)
    for row in read:
        rows.append(row[6])

indices = {}
for i in range(-1, 9):
  indices[i] = []

for i in range(len(labels)):
  indices[labels[i]].append(rows[i])

for item in indices:
  print(item)
  print(indices[item][0:20])

At this point, you can use the index numbers to look up the associated lightcurve plots to observe shape.